### Generating an Initial Start

In [2]:
import json

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

In [3]:
world = load_world(r"C:\Users\arnav\OneDrive\Desktop\ai_powered_dm\shared_data\Kyrethys.json")
kingdom = world['kingdoms']['Aerthys']
town = kingdom['towns']["Brindlemark"]
character = town['npcs']['Kaelin Darkshadow']

In [4]:
system_prompt = """You are an AI Game master. Your job is to create a 
start to an adventure based on the world, kingdom, town and character 
a player is playing as. 
Instructions:
You must only use 2-4 sentences \
Write in second person. For example: "You are Jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""
world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character: {character}
"""

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import transformers
import torch

model_id = 'meta-llama/Llama-3.2-3B-Instruct'

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map= 0,
)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": world_info + '\nYour Start:'},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    temperature = 1,
)


In [ ]:
outputs = outputs[0]["generated_text"][-1]
start = outputs['content']
print(start)
world['start'] = start


In [9]:
def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)
        
save_world(world,r"C:\Users\arnav\OneDrive\Desktop\ai_powered_dm\shared_data\Kyrethys_as_in_video.json")

### Creating the Main Action Loop

In [10]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""
    
    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    
    model_output = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    result = model_output[0]["generated_text"][-1]
    result = result['content']
    return result

In [11]:
game_state = {
    "world": world['description'],
    "kingdom": kingdom['description'],
    "town": town['description'],
    "character": character['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)